Loaded In the CSV's --> Converted to pandas dataframes --> assigned column headings

In [4]:
import pandas as pd
#pd.set_option("display.max_rows", 10, "display.max_columns", None, 'display.max_colwidth', None)
pd.set_option("display.max_rows", 10, "display.max_columns", None, 'display.max_colwidth', None, 'display.width', None)

dim = pd.read_csv("Dimensions.csv", header=None)
del dim[0]; del dim[1]
dim.columns = ['Painting ID', 'Dimensions']

loc = pd.read_csv("Locations.csv", header=None)
del loc[1]; del loc[4]
loc.columns = ['Painting ID', 'Location', 'Artist', 'Title']

Parse through Location CSV (has more info per id, must be assigned to Dimensions' painting IDs) --> create new merge dictionairy --> merge CSVs into singular dictionary via common id --> process differences in ID's (does result in loss of information although that is not a major concern) 

#
Note for improvement:
Searching algorithm is not optimized, and is actually very inefficient.
However, it is worth noting that this is not a major problem given the low size of Nasher's storage. Scaleability wise, given that Nasher currently does not seem to be expanding to millions of pieces anytime soon, optimization/minor improvements in speed for parsing TMS data is not a major concern
#

In [5]:
#comb = pd.DataFrame(data={'Painting ID':dim['Painting ID'], 'Dimensions':dim['Dimensions']})
combination = {}
for i in range(len(loc['Painting ID'])):
    if (loc['Painting ID'][i] in list(dim['Painting ID'])):
        combination[loc['Painting ID'][i]] = [loc['Location'][i], loc['Artist'][i], loc['Title'][i]]
    else:
        id_deconcat = loc['Painting ID'][i].split(' ')[0]
        id_deconcat = id_deconcat.split('.')
        for m in range(len(id_deconcat)-1, -1,-1):
            new_ID = id_deconcat[0]
            for j in range(1,m+1):
                new_ID = new_ID + f".{id_deconcat[j]}"
            if (new_ID in list(dim['Painting ID'])):
                combination[new_ID] = [loc['Location'][i], loc['Artist'][i], loc['Title'][i]]
                break
            if (m==0):
                quit(f"ERROR - PAINTING ID {loc['Painting ID'][i]} NOT FOUND")
    
for item in combination.keys():
    for i in range(len(list(dim['Painting ID']))):
        if (dim['Painting ID'][i] == item):
            combination[item].append(dim['Dimensions'][i])
            #print(combination[item])

converted merged dictionary to pandas dataframe --> introduced new rows where cleaned location data will be dropped into

In [6]:
painting_IDs = list(combination.keys())
locations = []
artist = []
title = []
dimensions = []
for item in painting_IDs:
    locations.append(combination[item][0])
    artist.append(combination[item][1])
    title.append(combination[item][2])
    dimensions.append(combination[item][3])
comb = pd.DataFrame(data={'Painting ID':painting_IDs,'Location':locations,'Artist':artist,'Title':title,'Dimension':dimensions})
comb.insert(2,"Location Type",None)
comb.insert(3,"Rack",None)
comb

,Painting ID,Location,Location Type,Rack,Artist,Title,Dimension
0,1966.50.1,"Nasher Museum Building, Nasher Painting Storage Room, Wall Screen, 27, Textile Roll # 25",None,None,Spanish,"Stenciled Leather Rug or Wall Covering, 18th century",152 x 88 1/2 inches (386.1 x 224.8 cm)
1,1967.2.1,"Nasher Museum Building, Nasher Painting Storage Room, Screen, 10B",None,None,attributed to Nicolaes Berchem,"Pastoral Scene, 17th century",30 1/8 x 41 3/4 inches (76.5 x 106 cm)
2,1969.10.1,"Nasher Museum Building, Nasher Painting Storage Room, Screen, 25B",None,None,George Benjamin Luks,"Miss Francis Lord Seymour, c. 1915",sight: 36 x 26 inches (91.4 x 66 cm)
3,1969.14.1,"Nasher Museum Building, Nasher Painting Storage Room, Screen, 54A",None,None,"Paul Jenkins, American, 1923 - 2012","Phenomena Above Hanks' Mirror, 1969",72 x 60 inches (182.9 x 152.4 cm)
4,1971.4.1,"Nasher Museum Building, Nasher Painting Storage Room, Screen, 25B",None,None,Francis Bicknell Carpenter,"Henry Ward Beecher, 1858",34 x 27 inches (86.4 x 68.6 cm)
...,...,...,...,...,...,...,...
781,2019.31.1,"Nasher Museum Building, Nasher Painting Storage Room",None,None,Mark Grotjahn,"Untitled (Brown Black and Red Spotted Heavy Texture Mask M16.e), 2012",43 1/2 × 16 × 5 1/4 inches (110.49 × 40.64 × 13.34 cm)
782,2020.2.1,"Nasher Museum Building, Nasher Painting Storage Room, Wall Screen, 52",None,None,David Hartt,"Lounge at The Johnson Publishing Company Headquarters, Chicago, Illinois, 2011",48 × 64 inches (121.92 × 162.56 cm)
783,2020.3.1,"Nasher Museum Building, Nasher Painting Storage Room, Wall Screen, 54",None,None,Hugo McCloud,"push pull, 2019",55 × 85 inches (139.7 × 215.9 cm)
784,2020.4.1,"Nasher Museum Building, Nasher Painting Storage Room, Wall Screen, 51",None,None,Fritz Scholder,"Indian and Storefront, 1974",Sight: 30 × 40 inches (76.2 × 101.6 cm)\r\nFrame: 32 1/4 × 42 1/2 × 2 inches (81.92 × 107.95 × 5.08 cm)


Cleaning Dimensions

In [7]:
def dimens(dim_string):
    print(dim_string,end='\n\n\n\n')
    dim_string = dim_string.split("\n")[0]
    return ("im stupid","im stupid","im stupid")

This is cleaning the locations and calling dimensions cleaner

In [8]:
for i in range(len(comb['Location'])):
    storageloc = comb['Location'][i].split(',')
    try:
        if ('SCREEN' in storageloc[2].strip().upper()):
            comb['Location Type'][i] = storageloc[2].strip().title()
        else:
            #print('\n',storageloc, comb['Painting ID'][i],'\n')
            pass
    except IndexError as E:
        '''print(storageloc, comb['Painting ID'][i])
        print(E)'''
        pass
    
    try:
        if(not ' ' in storageloc[3].strip()):
            comb['Rack'][i] = storageloc[3]
            
    except IndexError as E:
        pass
    
    (length, width, depth) = dimens(comb['Dimension'][i])
        
    

152 x 88 1/2 inches (386.1 x 224.8 cm)



30 1/8 x 41 3/4 inches (76.5 x 106 cm)



sight: 36 x 26 inches (91.4 x 66 cm)



72 x 60 inches (182.9 x 152.4 cm)



34 x 27 inches (86.4 x 68.6 cm)



Sight: 21 1/4 x 15 1/2 inches (54 x 39.4 cm)



37 1/16 x 53 1/16 inches (94.1 x 134.8 cm)



23 1/2 x 17 1/2 inches (59.7 x 44.5 cm)



26 1/2 x 40 inches (67.3 x 101.6 cm)



15 3/8 x 11 3/8 inches (39.1 x 28.9 cm)



27 1/2 x 21 1/2 inches (69.9 x 54.6 cm)



35 1/2 x 28 inches (90.2 x 71.1 cm)



approx.: 35 1/2 x 28 inches (90.2 x 71.1 cm)
Frame: 43 x 35 x 4 1/2 inches (109.2 x 88.9 x 11.4 cm)



plate: 9 3/16 x 13 1/8 inches (23.3 x 33.3 cm)
sheet: 13 x 17 7/8 inches (33 x 45.4 cm)



68 1/2 × 49 inches (174 × 124.5 cm)



nan





AttributeError: 'float' object has no attribute 'split'

In [25]:
pass